In [38]:
import numpy as np
import glob
import cv2
import sklearn
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, GlobalMaxPooling2D, Flatten, Dropout, GaussianNoise, Activation
import os, random
import shutil

In [39]:
train_generator = ImageDataGenerator(rescale=1/255,
                                    rotation_range=90,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.4,
                                    brightness_range=(0.1,0.9),
                                    channel_shift_range = 150,
                                    fill_mode="nearest")
                            
train = train_generator.flow_from_directory('/content/drive/MyDrive/number plates/train',target_size=(224,224),class_mode='categorical',shuffle=True)

Found 1995 images belonging to 2 classes.


In [40]:
val_generator = ImageDataGenerator(rescale=1./255)
val = val_generator.flow_from_directory('/content/drive/MyDrive/number plates/val',target_size=(224,224),class_mode='categorical',shuffle=False)

Found 450 images belonging to 2 classes.


In [42]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>=0.80):
            if(logs.get('val_acc')>=0.77):
                   print("\nReached 75.0% accuracy so cancelling training!")
                   self.model.stop_training = True

In [43]:
base_model = VGG16(weights='imagenet',include_top=False)

In [44]:
for layer in base_model.layers:
    layer.trainable = False

In [45]:
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = GaussianNoise(0.1)(x)
x = Dense(1280,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(500,activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(600)(x)
#x = GaussianNoise(0.1)(x)
#x = Activation('relu')(x)
#x = Dropout(0.2)(x)
#x = GaussianNoise(0.1)(x)
x = Dense(50,activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(2,activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = pred)

In [46]:
callbacks= myCallback()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
history = model.fit_generator(train,epochs=100,callbacks=[callbacks],validation_data = val)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
63/63 [==============================] - 41s 627ms/step - loss: 0.7905 - acc: 0.5071 - val_loss: 0.6622 - val_acc: 0.6667
Epoch 2/100
63/63 [==============================] - 39s 619ms/step - loss: 0.6942 - acc: 0.5638 - val_loss: 0.6822 - val_acc: 0.6667
Epoch 3/100
63/63 [==============================] - 39s 621ms/step - loss: 0.6805 - acc: 0.5948 - val_loss: 0.6610 - val_acc: 0.6667
Epoch 4/100
63/63 [==============================] - 39s 619ms/step - loss: 0.6674 - acc: 0.5817 - val_loss: 0.6819 - val_acc: 0.6089
Epoch 5/100
63/63 [==============================] - 39s 620ms/step - loss: 0.6495 - acc: 0.6157 - val_loss: 0.6360 - val_acc: 0.6667
Epoch 6/100
63/63 [==============================] - 39s 612ms/step - loss: 0.6492 - acc: 0.6076 - val_loss: 0.6603 - val_acc: 0.6000
Epoch 7/100
63/63 [==============================] - 39s 610ms/step - loss: 0.6197 - acc: 0.6646 - val_loss: 0.5691 - val_acc: 0.7089
Epoch 8/100
63/63 [==============================] - 39s 611ms

In [50]:
model.save('/content/drive/MyDrive/number plates/models/72valwith augumentation90')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/number plates/models/72valwith augumentation90/assets
